
- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [8]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import encode_tags

In [5]:
# load one file first to see what type of data you're dealing with and what attributes it has
import pandas as pd
pd.set_option('display.max_columns', 500)

#make list of file names in data folder

#loop through filenames in list 

#load json object
with open('../data/AK_Juneau_0.json') as f:
    nested_json = json.load(f)

def get_nested_value(data, *keys):
    try:
        for key in keys:
            data = data[key]
        return data
    except (TypeError, KeyError):
        return None

# Assuming your data is stored in 'response' variable
response_data = nested_json['data']['results']

# Initialize an empty list to store extracted values
extracted_data = []

for result in response_data:
    # Extract the 'name' from the 'branding' list, if available
    branding_name = nested_json['data']['results'][0]['branding'][0]['name']
    
    row = {
        'branding_name': nested_json['data']['results'][0]['branding'][0]['name'],  
        'community': get_nested_value(result, 'community'),
        'baths': get_nested_value(result, 'description', 'baths'),
        'baths_1qtr': get_nested_value(result, 'description', 'baths_1qtr'),
        'baths_3qr': get_nested_value(result, 'description', 'baths_3qr'),
        'baths_full': get_nested_value(result, 'description', 'baths_full'),
        'baths_half': get_nested_value(result, 'description', 'baths_half'),                              
        # Still need to add ('beds','garage','lot_sqft','name','sold_date','sold_price', 'sqft', 'stories', 'sub_type', 'type', 'year_built')
        
        'is_coming_soon': get_nested_value(result, 'flags', 'is_coming_soon'),
        'is_contingent': get_nested_value(result, 'flags', 'is_contingent'),
        # Still need to add ('is_for_rent', 'is_foreclosure', 'is_new_construction', 'is_new_listing', 'is_pending', 'is_plan', 'is_price_reduced', 'is_subdivision')
        
        'last_update_date': get_nested_value(result, 'last_update_date'),
        'show_contact_an_agent': get_nested_value(result, 'lead_attributes', 'show_contact_an_agent'),
        'list_date': get_nested_value(result, 'list_date'),
        'list_price': get_nested_value(result, 'list_price'),
        'listing_id': get_nested_value(result, 'listing_id'),
        'city': get_nested_value(result, 'location', 'address', 'city'),
        'lat': get_nested_value(result, 'location', 'address', 'coordinate', 'lat'),
        'lon': get_nested_value(result, 'location', 'address', 'coordinate', 'lon'),
        'line': get_nested_value(result, 'location', 'address', 'line'),
        'postal_code': get_nested_value(result, 'location', 'address', 'postal_code'),
        'state': get_nested_value(result, 'location', 'address', 'state'),
        'state_code': get_nested_value(result, 'location', 'address', 'state_code'),
        'open_houses': get_nested_value(result, 'open_houses'),
        'price_reduced_amount': get_nested_value(result, 'price_reduced_amount'),
        'brand_name': get_nested_value(result, 'products', 'brand_name'),
        'property_id': get_nested_value(result, 'property_id'),
        'status': get_nested_value(result, 'status'),
        'tags': get_nested_value(result, 'tags'),
    }
    extracted_data.append(row)

# Create a DataFrame from the extracted data
df = pd.DataFrame(extracted_data)
df.head()

,branding_name,community,baths,baths_1qtr,baths_3qr,baths_full,baths_half,is_coming_soon,is_contingent,last_update_date,show_contact_an_agent,list_date,list_price,listing_id,city,lat,lon,line,postal_code,state,state_code,open_houses,price_reduced_amount,brand_name,property_id,status,tags
0,EXP Realty LLC - Southeast Alaska,None,2.0,None,None,2.0,None,None,None,2023-09-19T20:52:50Z,True,2023-06-29T21:16:25.000000Z,554950.0,2957241843,Juneau,58.36395,-134.59372,9453 Herbert Pl,99801,Alaska,AK,None,45000.0,basic_opt_in,9074430767,sold,"[carport, community_outdoor_space, cul_de_sac,..."
1,EXP Realty LLC - Southeast Alaska,None,NaN,None,None,NaN,None,None,None,None,True,None,NaN,None,Juneau,NaN,NaN,8477 Thunder Mountain Rd,99801,Alaska,AK,None,NaN,None,9424983842,sold,None
2,EXP Realty LLC - Southeast Alaska,None,NaN,None,None,NaN,None,None,None,None,True,None,NaN,None,Juneau,NaN,NaN,4515 Glacier Hwy,99801,Alaska,AK,None,NaN,None,9479068516,sold,None
3,EXP Realty LLC - Southeast Alaska,None,NaN,None,None,NaN,None,None,None,None,True,None,NaN,None,Juneau,NaN,NaN,17850 Point Stephens Rd,99801,Alaska,AK,None,NaN,None,9879331943,sold,None
4,EXP Realty LLC - Southeast Alaska,None,NaN,None,None,NaN,None,None,None,None,True,None,NaN,None,Juneau,NaN,NaN,9951 Stephen Richards Memorial Dr,99801,Alaska,AK,None,NaN,None,9521639574,sold,None


At this point, ensure that you have all sales in a dataframe.
- Is each cell one value, or do some cells have lists?
- Maybe the "tags" will help create some features.
- What are the data types of each column?
- Some sales may not actually include the sale price.  These rows should be dropped.
- Some sales don't include the property type.
- There are a lot of None values.  Should these be dropped or replaced with something?

In [ ]:
# load and concatenate data here
# drop or replace values as necessary

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [ ]:
# OHE categorical variables here
# tags will have to be done manually

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE such as using central tendency?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- If you replace cities or states with numerical values, make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Drop columns that aren't needed.
- Don't keep the list price because it will be too close to the sale price.

In [ ]:
# perform train test split here
# do something with state and city
# drop any other not needed columns

**STRETCH**

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [ ]:
# import, join and preprocess new data here

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [ ]:
# perform EDA here

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized